In [ ]:
import scanpy as sc
import pandas as pd
import infercnvpy as cnv
import numpy as np 

import sys
import os 
sys.path.append(os.path.expanduser(f"~/SSS_mount/insituCNV/InSituCNV"))
import insitucnv as icv

# CosMx CNV inference

### Read in dataset
___

In [3]:
sid = '221'

In [4]:
adata = sc.read(os.path.expanduser(f"~/SSS_mount/insituCNV/data/WTx-CosMx_TVA/round2/{sid}/adata_{sid}_processed_1.h5ad"))
# adata = sc.read(os.path.expanduser(f"~/SSS_mount/insituCNV/data/WTx-CosMx_TVA/round2/data_from_borka/adata_{sid}_processed_1.h5ad"))

In [5]:
adata

AnnData object with n_obs × n_vars = 347023 × 18545
    obs: 'fov', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'Width', 'Height', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD68_CK8_18', 'Max.CD68_CK8_18', 'Mean.CD298_B2M', 'Max.CD298_B2M', 'Mean.CD45', 'Max.CD45', 'Mean.DAPI', 'Max.DAPI', 'cell_id', 'Dash', 'ISH.concentration', 'Panel', 'Run_Tissue_name', 'Run_name', 'assay_type', 'dualfiles', 'tissue', 'version', 'slide_ID', 'CenterX_global_px', 'CenterY_global_px', 'cell_ID', 'unassignedTranscripts', 'median_RNA', 'RNA_quantile_0.75', 'RNA_quantile_0.8', 'RNA_quantile_0.85', 'RNA_quantile_0.9', 'RNA_quantile_0.95', 'RNA_quantile_0.99', 'nCount_RNA', 'nFeature_RNA', 'median_negprobes', 'negprobes_quantile_0.75', 'negprobes_quantile_0.8', 'negprobes_quantile_0.85', 'negprobes_quantile_0.9', 'negprobes_quantile_0.95', 'negprobes_quantile_0.99', 'nCount_negprobes', 'nFeature_negprobes', 'median_falsecode', 'falsecode_quantile_0.75', 'falsecode_quantile_0.8', 'falsecode_quantil

### Run CNV inference and epithelial clustering
_______

In [ ]:
sc.pp.neighbors(adata, n_neighbors=2000)

In [ ]:
adata.uns['neighbors']

In [ ]:
neighbors = [2000] #50, 100, 200, 500, 1000, 

for n_neigh in neighbors:
    
    ad = adata.copy()
    ad.X = ad.layers['raw'].copy()
    sc.pp.normalize_total(ad)
    
    ad.layers['raw'] = ad.X.copy()
    icv.tl.smooth_data_for_cnv(ad, n_neighbors=n_neigh)
    ad.X = ad.layers['M'].copy()
    sc.pp.normalize_total(ad)
    sc.pp.log1p(ad)

    cnv.tl.infercnv(ad, 
                    window_size =100,
                    reference_key = 'ROI_status',
                    reference_cat = ['REF'],
                    chunksize = 1000)
    

    # Select only the epithelial cells
    adata_epi = ad[ad.obs.add_sub == 'epi'].copy()
    
    cnv.tl.pca(adata_epi)
    cnv.pp.neighbors(adata_epi)

    res=0.1
    
    # Cluster only the epithelial cells
    print('leiden res', res)
    key_added=f'epi_cnv_leiden_res{res}'
    cnv.tl.leiden(adata_epi, resolution = res, key_added=key_added)

    # Add the epithelial clustering back to the whole CosMx dataset
    ad.obs[key_added] = "non-epi"
    ad.obs.loc[adata_epi.obs_names, key_added] = adata_epi.obs[key_added]

    # Plot CNV heatmap with dentrogram and vmin and vmax for comparison.
    print('heatmap')
    save_heatmap = f"_CosMx_{sid}_smooth_n_{n_neigh}_groupby-{key_added}.pdf"
    cnv.pl.chromosome_heatmap(ad, groupby=key_added, dendrogram=True, save=save_heatmap, vmin=-.1) 

    sc.pl.spatial(ad, color=key_added, spot_size=100)

In [ ]:
adata